<a href="https://colab.research.google.com/github/Nagaraj-gt/sl1-quarterly-results-stock-predictor/blob/main/BSE_India_Company_Quarterly_Results_with_Stock_Labels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Details

**Name** : Indian companies quarterly results extractor along with labelled data for stock price impacted in next quarter

**Description** : Indian listed companies in stock exchange BSE/NSE need to publish quarterly results of their performace. This results impacts the stock price variations in next quarter. Investment companies vould leverage this tool to extract the quarterly results historically for strategic investments also. Also helps in realtime investment on company stocks based on its quarterly financial results. 





# Problem Description

The Investment Firm wants to invest in companies of a particular sector. The investment is on equities for 3 months period. Investment is based on performance metrics of the companies published in quarterly results. The Investment Company want to understand if the return on equities would bring minimum expected returns or not.


## Sector and Industries

For this problem Statement we have considered companies under FMCG Sector. 

As per [INDIAN INDUSTRY CLASSIFICATION](https://www.bseindia.com/Downloads1/India_Industry_Classification_Structure.pdf) by BSE and NSE to have common classification of companies, below are the basic industries under FMCG :

1.	Edible Oil

2.	Sugar

3.	Tea & Coffee

4.	Other Agricultural Products

5.	Breweries & Distilleries

6.	Other Beverages

7.	Cigarettes & Tobacco Products

8.	Animal Feed

9.	Dairy Products

10.	Other Food Products

11.	Packaged Foods

12.	Personal Care

13.	Household Products

14.	Batteries

15.	Photographic Products

16.	Stationary

17.	Diversified FMCG




# Data Sources

1. The quarterly financial statements published by the Indian FMCG companies. These are published in BSE and NSE websites.
2. The external factors which uniformly impacts all the companies irrespective of operational efficiency are tax and inflation. These are collected from Government wwebites 


# Data Extraction

## Extraction of quarterly finantial statements from BSE website

Common imports, defaults for formatting in Scrapping, Matplotlib, Pandas etc.

Also common functions to be used.



In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import datetime
import requests
from calendar import month_abbr

def removeSpecial(special_string):
  #special_string = special_string.replace('&nbsp;','_')
  return "".join((re.findall("[a-zA-Z\x20]+",special_string)))

## Common Constansts

headers = {
       'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    }

Extraction of collection of performance metrics data for a company listed in BSE

In [ ]:


# Function to retrieve quarterly results of a listed company from 2014 to 2019. As per BSE, quarter 81 is Jan 2014 to March 2014.
def getQuarter(i):

  if(i<81):
    return None

  base_year = 2014
  base_number = 81

  add_years, add_quarter = divmod((i-base_number),4)

  given_year = base_year + add_years
  given_qtr = add_quarter + 1

  return "Q"+ str(given_qtr) + " " + str(given_year)

def getQuarterlyResults(company_scode, company_name, qtr):
 
 URL = "https://www.bseindia.com/corporates/Results.aspx?Code=" + company_scode + "&Company="+company_name.replace(" ","%20") + "&qtr="+ str(qtr) +"&RType="
 
 try:
   page = requests.get(URL, headers=headers)
   soup = BeautifulSoup(page.content, "lxml") 
   #results = soup.find(id="ContentPlaceHolder1_tbl_typeID")

   # Getting all html tables in the page into dataframe.
   html_tables_df = pd.read_html(page.content)

   # Extracting The second HTML Table of the page which contains the company metadata like security code and name.
   company_table = html_tables_df[1].iloc[:1,[0,1,3,4]]
   
   security_code_title = removeSpecial(company_table.iloc[0,0] ).strip()
   company_name_title =  removeSpecial(company_table.iloc[0,2] ).strip()

   company_dict = { security_code_title :[company_table.iloc[0,1]] , company_name_title : [company_table.iloc[0,3]]  }
   company_df = pd.DataFrame(company_dict)

   # Extracting The fourth HTML Table of the page which contains the company quarterly financial statement
   
   
   quarterly_results_table =  html_tables_df[3].iloc[0:22,[0,1]]
   quarterly_results_table.columns = ['parameter','value']
   quarterly_results_table['parameter'] = quarterly_results_table['parameter'].transform(removeSpecial)

   # Transpose the dataframe to make column 0 values as column names
   index_ = quarterly_results_table.iloc[0:len(quarterly_results_table),[0]] 
   quarterly_results_table.index = index_

   quarterly_results_df = pd.DataFrame.transpose(quarterly_results_table.iloc[0:len(quarterly_results_table),[1]])
   
   quarterly_results_df.columns = [removeSpecial(col_name[0]) for col_name in quarterly_results_df.columns]
   
   
   if quarterly_results_df['Date Begin'] is None:
     print("The " + company_name + " has financial results only upto " + getQuarter(qtr+1) )
     return None

   company_df.index = quarterly_results_df.index

   ## Adding company name and code details.
   quarterly_results_df = company_df.join(quarterly_results_df)
   
   return quarterly_results_df
  
 except Exception as E:
   print(E)
   return None




def getAllQuarterResults(company_scode, company_name):
  ## Getting All quarters results from 2014 to 2019. Quarter 81 relates to Q1 2014

  all_quarters_results = []

  for i in range(104, 80, -1):
    quaterlyResults = getQuarterlyResults(company_scode, company_name, i)
    
    if(quaterlyResults is None):
      break
    else:
      quaterlyResults.insert(loc = 3,
          column = 'Quarter',
          value = getQuarter(i))
      
      all_quarters_results.append(quaterlyResults)
  
  all_quarter_results_df = pd.concat(all_quarters_results)
  all_quarter_results_df.reset_index(inplace=True, drop=True)

  return all_quarter_results_df
  



In [ ]:
# Function to label the quarterly results of a company with next quarter stocks deviations

def label_quaterly_results (quarterly_results_df):
  bse_stock_price_api = " https://api.bseindia.com/BseIndiaAPI/api/StockpricesearchData/w"
  
  stock_price_list = [] 
  for i in range(0,len(quarterly_results_df)) :
   
    month = pd.to_datetime(quarterly_results_df.loc[i,'Date Begin']).strftime(("%b"))
    for k, v in enumerate(month_abbr):
      if v == month:
          month = k
          break
    year = pd.to_datetime(quarterly_results_df.loc[i,'Date Begin']).strftime(("%Y"))
    
    #And +3, as we need the next quarter stocks to predict.
    month = month + 3
    next_quarter_begin = month if month <=12  else month%12

    # The BSE stocks API expected 0 to be pre-fixed for single digit months. 
    if(len(str(next_quarter_begin)) == 1):
      next_quarter_begin = "0" + str(next_quarter_begin)

    params = {
      'MonthDate': next_quarter_begin,
      'YearDate': year,
      'pageType':0,
      'rbType': 'M',
      'Scode': quarterly_results_df['Security Code']
    }

    api_result = requests.get(bse_stock_price_api, params, headers=headers)
    api_response = api_result.json()
    next_qtr_stocks_df = pd.DataFrame.from_dict(api_response.get('StockData'))
  
  
    # Calculation the stock value growth in next quarter
    
    qtr_open_price = float(next_qtr_stocks_df['qe_open'].iloc[0].replace(',',''))
    qtr_close_price = float(next_qtr_stocks_df['qe_close'].iloc[len(next_qtr_stocks_df) - 1].replace(',',''))

    next_qtr_stock_growth = qtr_close_price - qtr_open_price/ qtr_open_price
   
    stock_price_list.append(next_qtr_stock_growth)
  
  quarterly_results_df['Next Stock Deviation'] = stock_price_list
                                                                                                             

In [ ]:
## Function to consolidate all the labelled quartely results of companies belonging to a industry

def getAllLabelledQuaterlyResults(industry):

  listed_companies_url = "https://api.bseindia.com/BseIndiaAPI/api/ListofScripData/w"

  params = {
      'Group': 'A',
      'Scripcode': '',
      'industry':industry,
      'segment': 'Equity',
      'status': 'Active'
    }

  try:
    api_result = requests.get(listed_companies_url, params, headers=headers)
    api_response = api_result.json()
    
    
  
    frames = []

    for company in api_response:
      
      try:
        print("\n\tGetting 2014-19 quarterly returns for company " + company.get('Scrip_Name'))
        # Scraping the quarterly results 
        company_quarterly_results = getAllQuarterResults(company.get('SCRIP_CD') , company.get('Scrip_Name'))
       
        # Inserting industry

        industry_col = [industry] * len(company_quarterly_results)

        company_quarterly_results.insert(loc = 2,
            column = 'Industry',
            value = industry_col)
        
        print("\t\tLabelling the quarterly results of company " + company.get('Scrip_Name') + " with Next Quarter Stock Prices" )
        label_quaterly_results(company_quarterly_results)
        print("\t\tLabelling completed successfully" )
        frames.append(company_quarterly_results)
        print("\tTotal number of quarterly results received: (Expected : 24)  " + str(len(company_quarterly_results)))
      
      except Exception as E:
        print(E)

    return pd.concat(frames)

  except Exception as E:
   print(E)
   return None








## Retrieving the quarterly results for an industry

In [ ]:

# The labelled quarterly results of below Industry is 

fmcg_industries=['Edible Oil','Sugar','Tea & Coffee','Other Agricultural Products','Breweries & Distilleries','Other Beverages','Cigarettes & Tobacco Products','Animal Feed','Dairy Products','Other Food Products','Packaged Foods','Personal Care','Household Products','Batteries','Photographic Products','Stationary','Diversified FMCG']

fmcg_industries_list = []

for industry in fmcg_industries:
  print("\n\nCOLLECTING LABELLED DATA FOR  " + industry)
  labelled_listed_companies_df = getAllLabelledQuaterlyResults(industry)
  if labelled_listed_companies_df is not None :
    fmcg_industries_list.append(labelled_listed_companies_df)

# Collecting the common metrics across the basic industries under FMCG Sector for quarterly results
fmcg_companies_df = pd.concat(fmcg_industries_list,join='inner', ignore_index=True)


fmcg_companies_df.to_csv("Common_FMCG_Labelled_Quarter_Results.csv")
print("Successfully exported to CSV")
print(fmcg_companies_df.shape)







# Exploratory Data Analysis

## Analysis of FMCG Companies listed since Q1 2014 with Quarterly Data



In [ ]:
fmcg_companies_since_2014 = fmcg_companies_df[fmcg_companies_df['Quarter'] == 'Q1 2014']

print("Number of such companies are ", len(fmcg_companies_since_2014))

  

Number of such companies are  19


In [ ]:
print ("Below are list of such companies :")

print(fmcg_companies_since_2014['Company Name'])

Below are list of such companies :
31                 BALRAMPUR CHINI MILLS LTD.
55     Dalmia Bharat Sugar and Industries Ltd
113           DWARIKESH SUGAR INDUSTRIES LTD.
137                  SHREE RENUKA SUGARS LTD.
229               GUJARAT AMBUJA EXPORTS LTD.
284                   UNITED BREWERIES LTD.-$
308                       RADICO KHAITAN LTD.
387                       AVANTI FEEDS LTD.-$
421                  HATSUN AGRO PRODUCT LTD.
445                         NESTLE INDIA LTD.
492                  TASTY BITE EATABLES LTD.
516                        VENKY S INDIA LTD.
591                                   PROCTER
615              COLGATE-PALMOLIVE INDIA LTD.
639                       GILLETTE INDIA LTD.
697                   Bajaj Consumer Care Ltd
755            EVEREADY INDUSTRIES INDIA LTD.
800                                  ITC LTD.
824                      HINDUSTAN FOODS LTD.
Name: Company Name, dtype: object


The Analysis says that there are only **19** FMCG companies with data since 2014. For our model we need **20** companies. From simple excel output we understood that there is one more company which has data in 2014 but not since Q1. The company is **HINDUSTAN UNILEVER LTD.**. The first 3 quarters of 2014 has missing data due to point failures from BSEINDIA website for those requests.

Lets add HINUDSTAN UNILEVER LTD. company data as well and fill the quarterly results for  HINUDSTAN UNILEVER LTD data for first 3 quarters of 2014 from moneycontrol.com

### Creating the DataFrame based on above listed companies for modelling



In [ ]:
fmcg_companies_list = (fmcg_companies_df[fmcg_companies_df['Quarter'] == 'Q4 2014']['Company Name']).to_list()

mask = fmcg_companies_df['Company Name'].isin(fmcg_companies_list)

fmcg_companies_labelled_results_df = fmcg_companies_df.loc[mask]

display(fmcg_companies_labelled_results_df)

fmcg_companies_labelled_results_df.to_csv("FMCG_Companies_Data_2014.csv")

## Data Cleanup



In [ ]:
# Remove uncessary information columns
fmcg_companies_labelled_results_df.reset_index(inplace=True, drop=True)

if 'Description' in fmcg_companies_labelled_results_df.columns:
  fmcg_companies_labelled_results_df.pop('Description')





<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 29 columns):
 #   Column                                                                     Non-Null Count  Dtype  
---  ------                                                                     --------------  -----  
 0   Security Code                                                              477 non-null    int64  
 1   Company Name                                                               477 non-null    object 
 2   Industry                                                                   477 non-null    object 
 3   Type                                                                       477 non-null    object 
 4   Quarter                                                                    477 non-null    object 
 5   Date Begin                                                                 477 non-null    object 
 6   Date End                                                  

In [ ]:
# Checking Data
fmcg_companies_labelled_results_df.Industry.value_counts()


Sugar                          96
Personal Care                  96
Packaged Foods                 72
Diversified FMCG               69
Breweries & Distilleries       48
Other Agricultural Products    24
Animal Feed                    24
Dairy Products                 24
Batteries                      24
Name: Industry, dtype: int64

The data records are spread across various industries under FMCG Sector as above.

In [ ]:
fmcg_companies_labelled_results_df.Industry.value_counts(normalize=True)*100

Sugar                          20.125786
Personal Care                  20.125786
Packaged Foods                 15.094340
Diversified FMCG               14.465409
Breweries & Distilleries       10.062893
Other Agricultural Products     5.031447
Animal Feed                     5.031447
Dairy Products                  5.031447
Batteries                       5.031447
Name: Industry, dtype: float64